In [1]:
!pip install -qqq autogen groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

GROQ_API_KEY=userdata.get('GROQ_API_KEY')

In [4]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

temp_dir=tempfile.TemporaryDirectory()
config_list = [
    {
        # Let's choose the Llama 3 model
        "model": "llama-3.2-90b-text-preview", #llama-3.2-90b-text-preview

        # Put your Groq API key here or put it into the GROQ_API_KEY environment variable.
        "api_key": GROQ_API_KEY,
        # We specify the API Type as 'groq' so it uses the Groq client class
        "api_type": "groq",
    }
]

executer=LocalCommandLineCodeExecutor(work_dir=temp_dir.name)


In [35]:

code_writer_system_message = """
You need to calculate:

1- The number of packs of each ingredient to buy, ensuring total cost is within the budget with respect to consentraints.
2- The number of units of each product to sell, maximizing profit above inventory costs with respect to consentraints.

Constraints:

1- Budget Constraint:
    The total cost of ingredients must not exceed the budget:

    buying_price1 * pack_count1 + buying_price2 * pack_count2 + ... + buying_price34 * pack_count34 <= Budget=2500

    - So when you are calculating pack_counts, results must always work in the above equation.

2- Profit Constraint:

    Profit after deducting inventory costs must be positive:

    ((sell_price1 * number_of_selling1 + sell_price2 * number_of_selling2 + ... + sell_price40 * number_of_selling40)
     - (buying_price1 * pack_count1 + buying_price2 * pack_count2 + ... + buying_price34 * pack_count34)) > 0

3- If the total number of selling products is for example 100, with respect to sell_ration 10 of them are Espresso. Keep watch the sale_ratio.
  There are 2.5% and float number in sell_ratio too
4- Minimum Pack Count:
    Each ingredient must have a minimum pack count of 1 to ensure stock availability.

Dataset Structure:

You will work with a JSON dataset with 40 products, each containing the following structure:

{ "products": [ { "name": "Espresso", "sell_price": 1.40, "sale_ratio": "10%", "ingredients": [ {"name": "Espresso beans", "quantity_usedIn_product_per_buying_unit": 22, "unit": "grams", "buying_price": 17.3, "buying_unit": 1000}, {"name": "Water", "quantity_usedIn_product_per_buying_unit": 60, "unit": "ml", "buying_price": 0.5, "buying_unit": 1500} ] }}


- The above is just an example. There are 40 products.

Each product has:

sell_price: The price at which the product is sold. (Float number)
sale_ratio: Expected sales percentage in relation to other products. (it is in "10%" foramt or "Float %")
ingredients: List of ingredients required, with each ingredient containing:
    unit: Unit of measurement for the ingredient. (it is not important in this calculation)
    quantity_usedIn_product_per_buying_unit: Amount needed of each ingredients per buying_unit. (Float number)
    buying_price: Price per unit pack.
    buying_unit: Pack size for the ingredient which is bought.

Instructions:

1- Calculate Ingredient Packs:
    For each of the 34 ingredients, determine the  number of packs required based on the products’ demands and the pack sizes.
    Just use buying_price for calculating the pack count.
    Ensure that the total ingredient cost, which is demonstrained in instruction1, does not exceed the specified budget. It should be close to the budget
    Each ingredient’s pack count must be at least 1.

2- Calculate Product Quantities:
    Based on the sale_ratio and sell_price, calculate the optimal quantity of each product to sell to maximize profit.
    Ensure that the total profit minus inventory cost is positive, as per the profit constraint.
    * Important: The sale_ratio represents each product’s target proportion of total sales. For example, if the sale_ratio of "Espresso" is 10%, then out of every 100 products sold, 10 should ideally be Espresso. Use this information to guide the relative quantities of each product sold.

    _ create a function to check each constraints are satisfied.

3- Output Format:
    Provide the result as a JSON output with ingredient pack counts and product sale quantities in the following format:

    {
      "ingredients": {
        "Ingredient1": PackCount1,
        "Ingredient2": PackCount2,
        ...
        "Ingredient34": PackCount34
      },
      "products": {
        "Product1": SellCount1,
        "Product2": SellCount2,
        ...
        "Product40": SellCount40
      }
    }

    Include all 34 ingredients with their calculated pack counts (no omissions).
    Include all 40 products with their calculated quantities to sell.

Additional Guidelines:
    * Ensure that pack counts and sale counts are integers with no zero values.
    * Distribute the budget efficiently to avoid purchasing excess packs where unnecessary.
    * Do not buying_unit in your calculation. Just use buying_priice for calculating the inventory cost.


Example Task Flow:

- Read the JSON file from the given path.
- Calculate the required pack counts for each ingredient, based on buying_price and adhering to the budget.
- For calculating the required pack counts, find the best numbers to make the output close to budget, which is 2500.
- Calculate the selling quantities for each product, ensuring profitability based on given sell prices.
- The numbers should be rational. If the sale_ratio of a product is high, it must be sold more.
- Ensure that pack counts and sale counts are integers with no zero values.
- Output the optimized inventory and sales plan in JSON format.

Execution Requirements:

- Provide the complete code in a single Python code block for direct execution.
- Verify the output format matches the required JSON structure precisely.
- Do not make any assumptions or fabricate values; use only the data provided in the JSON.
"""

In [36]:

code_writer_agent=ConversableAgent(
    'code_writer',
    llm_config={"config_list": config_list},
    system_message=code_writer_system_message,
    code_execution_config=False

)

code_executer_agent=ConversableAgent(
    'code_executer',
    llm_config=False,
    code_execution_config={'executor':executer},
    human_input_mode='ALWAYS',
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"]
)


In [37]:

chat_result = code_executer_agent.initiate_chat(
    code_writer_agent,
    message=f"Here is the path: /content/Full_Dataset.json, save the JSON to a file named 'inventory_product.json. this code must satisfied all of the consentraints And the output must be rationale. Think step bu step and accurracy is more important",

)

code_executer (to code_writer):

Here is the path: /content/Full_Dataset.json, save the JSON to a file named 'inventory_product.json. this code must satisfied all of the consentraints And the output must be rationale. Think step bu step and accurracy is more important

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer (to code_executer):

Below is a Python code block that meets the requirements and should work with the provided JSON file. It's a step-by-step process, and accuracy has been prioritized.

```python
import json

# Path to the input JSON file
input_file_path = '/content/Full_Dataset.json'

# Budget constraint
budget = 2500

# Load the JSON data
with open(input_file_path) as f:
    data = json.load(f)

# Extract required data for calculations
products = data['products']
ingredients_with_prices = {}

# Populate the dictionary of ingredients with their buying prices
for product in products:
    for ingred

/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.2-90b-text-preview
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Replying as code_executer. Provide feedback to code_writer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executer (to code_writer):

exitcode: 1 (execution failed)
Code output: Traceback (most recent call last):
  File "/tmp/tmpujqzg3po/tmp_code_b6b9ab2ebac9ccc682077b874402f68e.py", line 74, in <module>
    raise ValueError("Total sale ratio is not 100%")
ValueError: Total sale ratio is not 100%


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer (to code_executer):

This error is caused by the total sale ratio not being equal to 1 (100%). This could be due to several reasons such as:

* Not all products' sale ratios add up to 100%.
* There may be some products with sale ratios that are not correctly formatted (e.g., missing '%' sign).
* There may

/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.2-90b-text-preview
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Replying as code_executer. Provide feedback to code_writer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executer (to code_writer):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.2-90b-text-preview` in organization `org_01j6ej5c64f61ahg90d2ptg60w` on tokens per minute (TPM): Limit 7000, Requested 7555, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [38]:
import os

print(os.listdir(temp_dir.name))

['tmp_code_5109ebd1a9dadcbb31804726f4b7914a.py', 'tmp_code_38f5f0d4f1d8a8a973200f53c6d978eb.py', 'tmp_code_4ec0be2372f7129da791497ec7a7bc3e.py', 'tmp_code_6d790bec131ed401d2a6fe3ca00f82de.py', 'inventory_product.json', 'tmp_code_fe2d428d8b430acc35579e88099bdfef.py', 'tmp_code_b6b9ab2ebac9ccc682077b874402f68e.py', 'tmp_code_d0c24a7dab58e187014de7cc4bad648c.py', 'tmp_code_9703e4aada0a368de6328e63e38437a8.py', 'tmp_code_5c8b869e2333588d8bd57570024012fc.py', 'tmp_code_072b1031a19a4fc1598d635585303601.py', 'tmp_code_949d7cfec8a0b7e004b585a4173c1ab8.py', 'tmp_code_bd66846679d240d8fd9b495cd4f93b72.py', 'tmp_code_47a2607808f8a4af5d6c64a435f81a56.py', 'tmp_code_8e0aee5008585fd970f5848a3a164170.py', 'tmp_code_c55b37bf89bbd534876ddf007df0daf1.py', 'tmp_code_307b8ef7fef03c9285670d99570f7cd6.py', 'tmp_code_1a3daa5efe60acff56f2c562b9e0d038.py', 'tmp_code_833c0215f57d0572cce616660a92ec51.py', 'tmp_code_cac5283160607fdda9c855e01d810040.py', 'tmp_code_71535213db5b20ed6a107d5af4976649.py']


In [39]:
import json

file=os.path.join(temp_dir.name,'inventory_product.json')
with open(file, 'r') as f:
    data = json.load(f)

In [40]:
data

{'ingredients': {'Espresso beans': 15,
  'Water': 101,
  'Espresso': 71,
  'Milk': 71,
  'Chocolate syrup': 7,
  'Ice-cream': 20,
  'Caramel sauce': 10,
  'Black tea loose leaf': 32,
  'Green tea loose leaf': 16,
  'Dried hibiscus flowers': 2,
  'Dried mint leaf': 14,
  'Honey': 29,
  'Dried ginger root': 3,
  'All-purpose flour': 14,
  'Sugar': 31,
  'Cocoa powder': 7,
  'Baking powder': 29,
  'Baking soda': 10,
  'Salt': 31,
  'Eggs': 27,
  'Vegetable oil': 31,
  'Vanilla extract': 7,
  'Butter': 26,
  'Lemon juice': 31,
  'Ground cinnamon': 6,
  'Ground ginger': 5,
  'Grated carrots': 8,
  'Cream': 17,
  'Strawberries': 13,
  'Graham cracker crumbs': 6,
  'Cream cheese': 23,
  'Banana': 29,
  'Yogurt': 29,
  'Mango': 12,
  'Peanut butter': 2},
 'products': {'Espresso': 98,
  'Americano': 39,
  'Latte': 1,
  'Cappuccino': 1,
  'Macchiato': 1,
  'Mocha': 1,
  'Flat white': 1,
  'Affogato': 1,
  'Caramel macchiato': 1,
  'Misto': 1,
  'Cortado': 1,
  'Black tea': 78,
  'Green tea': 42,

In [41]:
with open('inventory__profit_sell_ratio_optimization.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)